In [ ]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass
!pip install triton transformers
!pip install -U datasets
!pip install --pre -U xformers ##### this take some time

# ****Note** Restart the Kernal after package installation**

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_token")

In [2]:
from unsloth import FastLanguageModel
import torch
from IPython.display import display_markdown
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/llama-3-8b-bnb-4bit",  
]  #### loadin llama 3 model in 4 bit to fine tune

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
)

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.dev778. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-04-19 15:49:14.219228: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-19 15:49:14.219343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-19 15:49:14.372882: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


ref_config https://github.com/pytorch/torchtune/blob/main/recipes/configs/llama3/8B_qlora_single_device.yaml

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
#### dataset import 
dataset_name = "VMware/open-instruct"
from datasets import load_dataset
dataset = load_dataset(dataset_name, split="train")

Generating train split:   0%|          | 0/142622 [00:00<?, ? examples/s]

In [6]:
# Must add EOS_TOKEN at response last line
EOS_TOKEN = tokenizer.eos_token 
def mapping_response(sample):
    sample['text'] = sample['alpaca_prompt']+"\n"+sample['response']+EOS_TOKEN
    return sample

In [7]:
### selecting only 2000 samples for testing
dataset = dataset.select(range(2000))

In [8]:
dataset = dataset.map(mapping_response)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
def prompt_inference(prmpt):
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        prmpt
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
    return tokenizer.batch_decode(outputs)[0].split("### Response:")[-1]

In [10]:
prompt = dataset['alpaca_prompt'][90]
print(prompt)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Generate the lyrics to a song that begins like this "Hey hey let's go..."

### Response:


In [11]:
print("result")
display_markdown(prompt_inference(prmpt=prompt),raw=True)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result


 
Hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey hey let's go, hey

In [ ]:
### the above result is not clear ,without fine tuning

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text", ### taking text column from the dataset
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,
        warmup_steps = 500,
        max_steps = 800,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 100,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",num_train_epochs =1
    ),
)

Map (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [13]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 2
\        /    Total batch size = 2 | Total steps = 800
 "-____-"     Number of trainable parameters = 41,943,040
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
100,1.475300
200,1.231300
300,1.176200
400,1.221700
500,1.197700
600,1.234700
700,1.197600
800,1.211500


TrainOutput(global_step=800, training_loss=1.2432494068145752, metrics={'train_runtime': 1679.8268, 'train_samples_per_second': 0.952, 'train_steps_per_second': 0.476, 'total_flos': 2.051551084444877e+16, 'train_loss': 1.2432494068145752, 'epoch': 0.8})

In [ ]:
prompt = dataset['alpaca_prompt'][92]
print(prompt)

In [15]:
print("result")
display_markdown(prompt_inference(prmpt=prompt),raw=True)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result


 
Hey hey let's go
We're gonna have a good time
We're gonna make some noise
We're gonna have a blast
Hey hey let's go
We're gonna sing and dance
We're gonna make some memories
We're gonna have a blast
Hey hey let's go
We're gonna have a party
We're gonna make some friends
We're gonna have a blast
Hey hey let's go
We're gonna have a good time
We're gonna make some noise
We're gonna have a blast<|end_of_text|>